In [1]:
import pandas as pd
import numpy as np

In [2]:
import pickle

In [3]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [4]:
from sklearn.metrics import auc, roc_auc_score, roc_curve, accuracy_score,  recall_score, precision_score
from regression_tools.dftransformers import (
    ColumnSelector, Identity, FeatureUnion, MapFeature, Intercept)

In [5]:
from my_metr import plot_roc_curve

In [54]:
from trans_rat import FillPrice, CategoriesTransformScr, RatingNormalizer, FillPriceScr


In [6]:
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
from pymongo import MongoClient
import time
from bs4 import BeautifulSoup

mc = MongoClient()
db  = mc['raw_restaurants']
rv_s = db['reviews_scrap']

In [8]:
colClust_n = ['cl_t0', 'cl_t1', 'cl_t2', 'cl_t3', 'cl_t4', 'cl_t5',  'cl_t7',
       'cl_t8', 'cl_t9', 'cl_t10', 'cl_t11', 'cl_t12', 'cl_t13', 'cl_t14',
        'cl_t16', 'cl_t17']

In [ ]:
#numbers_list = pickle.load(list_unpickle)

In [9]:
clust_model_rf ={}
clust_model_gr = {}

In [10]:
for col in colClust_n:
    filename_gr = 'grboost/mgr_'+col+'.pkl'
    with open(filename_gr,'rb') as f:
        clust_model_gr[col]= pickle.load( f)
    filename_rf = 'rforest/mrf_'+col+'.pkl'
    with open(filename_rf,'rb') as f:
        clust_model_rf[col] =pickle.load( f)

In [11]:
clust_model_gr['cl_t3']

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.05, loss='deviance', max_depth=3,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=300,
              presort='auto', random_state=179, subsample=0.8, verbose=0,
              warm_start=False)

In [12]:
user_cl = pd.read_csv('user_from_scr_clidf18.csv')

In [13]:
rest = db['restaurants']

rest_df = pd.DataFrame(list(rest.find({})))

In [24]:
rest_df.shape

(8946, 18)

### надо почистить дату

In [14]:
revs_df = pd.DataFrame(list(rv_s.find({})))

In [16]:
revs_df.columns

Index(['_id', 'alias', 'biz_id', 'category', 'date', 'id', 'rating', 'text',
       'user_id'],
      dtype='object')

In [19]:
revs_df.drop(columns=[ '_id', 'category', 'date'], inplace= True)

In [21]:
rest_alias = np.unique(revs_df.alias)

In [22]:
len(rest_alias)

10222

In [27]:
act = (revs_df.rating.astype(float) < 3)

In [28]:
rest_df.head(3)

,_id,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,rev_API,review_count,transactions,url
0,5bf90d4e7988471b045ba501,maono-seattle,"[{'alias': 'hawaiian', 'title': 'Hawaiian'}, {...","{'latitude': 47.5636486287425, 'longitude': -1...",(206) 935-1075,1501.527536,gDAO-4m63svPUpii5b8kww,https://s3-media4.fl.yelpcdn.com/bphoto/rwJG59...,False,"{'address1': '4437 California Ave SW', 'addres...",Ma'ono,+12069351075,$$,4.0,1,788,[],https://www.yelp.com/biz/maono-seattle?adjust_...
1,5bf90d4e7988471b045ba502,ohs-sandwiches-seattle-2,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...","{'latitude': 47.5751165, 'longitude': -122.387...",(206) 420-3692,2616.713742,xzLAynrVb3No3ADUhMxlnQ,https://s3-media2.fl.yelpcdn.com/bphoto/zKlK95...,False,"{'address1': '3217 California Ave SW', 'addres...",Oh's Sandwiches,+12064203692,NaN,4.5,1,32,[],https://www.yelp.com/biz/ohs-sandwiches-seattl...
2,5bf90d4e7988471b045ba503,marination-ma-kai-seattle,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...","{'latitude': 47.5895547295386, 'longitude': -1...",(206) 328-8226,4061.556690,IsvlaFF0EPEhh3ahTuC4xg,https://s3-media1.fl.yelpcdn.com/bphoto/r19fwm...,False,"{'address1': '1660 Harbor Ave SW', 'address2':...",Marination Ma Kai,+12063288226,$$,4.0,1,897,[],https://www.yelp.com/biz/marination-ma-kai-sea...


In [29]:
rest_clean = rest_df.drop(columns= ['_id','coordinates', 'display_phone', 
                                    'distance','image_url', 'name','location',
                                   'is_closed','phone', 'transactions','url'])

In [31]:
rest_clean.head(3)

,alias,categories,id,price,rating,rev_API,review_count
0,maono-seattle,"[{'alias': 'hawaiian', 'title': 'Hawaiian'}, {...",gDAO-4m63svPUpii5b8kww,$$,4.0,1,788
1,ohs-sandwiches-seattle-2,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",xzLAynrVb3No3ADUhMxlnQ,NaN,4.5,1,32
2,marination-ma-kai-seattle,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",IsvlaFF0EPEhh3ahTuC4xg,$$,4.0,1,897


In [35]:
#Pipeline = Pipeline([FillPriceScr, CategoriesTransformScr, RatingNormalizer ])


In [93]:
fpr = FillPriceScr()

In [94]:
rest_cl2 = fpr.transform(rest_clean)
rest_cl2.head(2)

/Users/olga/yelper/trans_rat.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  X[self.priceCol][pd.isnull(X[self.priceCol])] = '$$'


,alias,categories,id,RestaurantsPriceRange2,rating,rev_API,review_count,rest_isna
0,maono-seattle,"[{'alias': 'hawaiian', 'title': 'Hawaiian'}, {...",gDAO-4m63svPUpii5b8kww,2,4.0,1,788,False
1,ohs-sandwiches-seattle-2,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",xzLAynrVb3No3ADUhMxlnQ,2,4.5,1,32,True


In [59]:
##Acai Bowls
mod_cat = pd.read_csv('model_cat.csv')
mod_cat.head(3)

,Unnamed: 0,0
0,Acai Bowls,0.000840
1,Active Life,0.004093
2,Afghan,0.002064


In [67]:
mod_fe = mod_cat.iloc[:,0].tolist()
rest_cl2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8946 entries, 0 to 8945
Data columns (total 8 columns):
alias           8946 non-null object
categories      8946 non-null object
id              8946 non-null object
price           8946 non-null int64
rating          8946 non-null float64
rev_API         8945 non-null object
review_count    8946 non-null int64
rest_isna       8946 non-null bool
dtypes: bool(1), float64(1), int64(2), object(4)
memory usage: 498.0+ KB


In [95]:
ct = CategoriesTransformScr(mod_fe)
rn = RatingNormalizer(starCol = 'rating')
rest_cx = rn.transform(ct.transform(rest_cl2))
rest_cx.head()

,alias,id,RestaurantsPriceRange2,stars,rev_API,review_count,rest_isna,Acai Bowls,Active Life,Afghan,...,Venues & Event Spaces,Vietnamese,Waffles,Whiskey Bars,Wine & Spirits,Wine Bars,Wineries,Wraps,num_of_categ,st_over_am
0,maono-seattle,gDAO-4m63svPUpii5b8kww,2,4.0,1,788,False,False,False,False,...,False,False,False,False,False,False,False,False,2,0.016000
1,ohs-sandwiches-seattle-2,xzLAynrVb3No3ADUhMxlnQ,2,4.5,1,32,True,False,False,False,...,False,False,False,False,False,False,False,False,2,0.140621
2,marination-ma-kai-seattle,IsvlaFF0EPEhh3ahTuC4xg,2,4.0,1,897,False,False,False,False,...,False,False,False,False,False,False,False,False,3,0.016000
3,ittos-tapas-seattle-4,Dr9CxgdePozHGwU4beaYGQ,2,4.5,1,221,False,False,False,False,...,False,False,False,False,False,False,False,False,3,0.020362
4,pizzeria-credo-seattle,BeGo3TSuhF7fOv0dzp0s1g,2,4.5,1,405,False,False,False,False,...,False,False,False,False,False,False,False,False,2,0.018000


In [96]:
### i forgot to include review_count to model when train
colPred1 = rest_cx.columns[2:4].union( rest_cx.columns[6:])

In [88]:
with open('colPred213', 'rb') as f:
    colPred = pickle.load(f)

In [97]:
colPred[colPred != colPred1]

Index([], dtype='object')

In [98]:
colPred1[colPred != colPred1]

Index([], dtype='object')

In [99]:
colPred1

Index(['Acai Bowls', 'Active Life', 'Afghan', 'African', 'American (New)',
       'American (Traditional)', 'Arabian', 'Arcades', 'Argentine',
       'Art Galleries',
       ...
       'Waffles', 'Whiskey Bars', 'Wine & Spirits', 'Wine Bars', 'Wineries',
       'Wraps', 'num_of_categ', 'rest_isna', 'st_over_am', 'stars'],
      dtype='object', length=213)

In [100]:
X = rest_cx[colPred]

,Acai Bowls,Active Life,Afghan,African,American (New),American (Traditional),Arabian,Arcades,Argentine,Art Galleries,...,Wine & Spirits,Wine Bars,Wineries,Wraps,num_of_categ,price,rest_isna,review_count,st_over_am,stars


In [101]:
clt0 = clust_model_rf['cl_t0'].predict_proba(X)[:,1]

In [102]:
clt0

array([0.67499515, 0.2146956 , 0.71224975, ..., 0.21640103, 0.15179362,
       0.42946443])

In [85]:
r0 = clust_model_rf['cl_t0']

## Restaurants Prediction
make 2 dictionary, then 2 dataframe with cluster predictions

In [106]:
%%time
Y_rf = pd.DataFrame()
Y_gr = pd.DataFrame()
Y_rf['alias'] = rest_cx['alias']
Y_gr['alias'] = rest_cx['alias']
for col in colClust_n:
    Y_rf[col] = clust_model_rf[col].predict_proba(X)[:,1]
    Y_gr[col] = clust_model_gr[col].predict_proba(X)[:,1]


CPU times: user 13.3 s, sys: 1.17 s, total: 14.4 s
Wall time: 9.24 s


In [132]:
Y_gr.head(3)

,alias,cl_t0,cl_t1,cl_t2,cl_t3,cl_t4,cl_t5,cl_t7,cl_t8,cl_t9,cl_t10,cl_t11,cl_t12,cl_t13,cl_t14,cl_t16,cl_t17
0,maono-seattle,0.929968,0.992710,0.412188,0.016085,0.837059,0.919372,0.010737,0.035540,0.050482,0.009742,0.096255,2.309888e-03,0.074948,0.0,0.272277,0.037797
1,ohs-sandwiches-seattle-2,0.172442,0.302586,0.010211,0.004146,0.025425,0.049360,0.001897,0.002025,0.005802,0.000076,0.005502,1.293178e-09,0.001371,0.0,0.005165,0.000869
2,marination-ma-kai-seattle,0.953005,0.995034,0.428013,0.003022,0.780916,0.921224,0.004737,0.012592,0.024337,0.342402,0.059423,7.697930e-04,0.050621,0.0,0.290787,0.010165


In [138]:
Y_gr['rating'] = rest_cx.stars
Y_rf['rating'] = rest_cx.stars


In [108]:
user_cl.head()

,Unnamed: 0,user_id,user_cl
0,0,MVb93PtUPbSsVFa-3prvcg,2
1,1,6XwhYkbgsPZNQLLvTWIc3Q,2
2,2,Re5H1w9VR-YqlKL-ydOlOg,0
3,3,havBggvuzfE6IPfkT-njbg,1
4,4,_efP61KpqeC9K4TX0aqwyw,2


In [115]:
user_cl.shape, user_cl.columns


((385, 3), Index(['Unnamed: 0', 'user_id', 'user_cl'], dtype='object'))

In [ ]:
for col in colClust_n:
    i = int(col[4:])
    u_col = 'u'+col
    user_cl[u_col] = (user_cl['user_cl']==i)
 

In [124]:
user_cl.head()

,Unnamed: 0,user_id,user_cl,ucl_t0,ucl_t1,ucl_t2,ucl_t3,ucl_t4,ucl_t5,ucl_t7,ucl_t8,ucl_t9,ucl_t10,ucl_t11,ucl_t12,ucl_t13,ucl_t14,ucl_t16,ucl_t17
0,0,MVb93PtUPbSsVFa-3prvcg,2,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
1,1,6XwhYkbgsPZNQLLvTWIc3Q,2,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
2,2,Re5H1w9VR-YqlKL-ydOlOg,0,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,3,havBggvuzfE6IPfkT-njbg,1,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,4,_efP61KpqeC9K4TX0aqwyw,2,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False


In [123]:
   
user_cl_n = user_cl.drop(columns=['Unnamed: 0','user_cl'])
user_cl_n = user_cl_n.groupby('user_id').sum()
user_cl_n.head()

,ucl_t0,ucl_t1,ucl_t2,ucl_t3,ucl_t4,ucl_t5,ucl_t7,ucl_t8,ucl_t9,ucl_t10,ucl_t11,ucl_t12,ucl_t13,ucl_t14,ucl_t16,ucl_t17
user_id,,,,,,,,,,,,,,,,
-KlnVOvjklQn5OpTbxFGzA,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
-Yb1srF4RYtpn-6Yp8ojRg,True,True,True,False,True,False,True,False,True,False,True,False,False,False,False,False
1SVDSLiWxbWmBjenfDQkbA,False,True,False,False,True,True,False,False,False,False,False,False,False,False,False,False
1Vq3xObhRXBU8gnPIN9jsQ,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2wdhN8Y2EJRswvkduFxnOQ,True,True,True,False,False,False,False,False,True,False,False,False,False,False,False,True


In [127]:
user_cl_n.shape

(99, 16)

In [112]:
revs_df.head()

,alias,id,rating,user_id
0,srivilai-thai-cuisine-seattle,ErsYo_9v8--feFCy92tymQ,5.0,_3KTfz0hcPl_yq4jOyOPiA
1,ohs-sandwiches-seattle-2,W1NnSItt-SwWHrzkRtZ44g,5.0,_3KTfz0hcPl_yq4jOyOPiA
2,saison-san-francisco-2,3Q5uzROHYl1i2B_t2sIl7w,3.0,_3KTfz0hcPl_yq4jOyOPiA
3,international-smoke-san-francisco-2,-GYe-Aq7NcNJ3u-U6fmwMg,5.0,_3KTfz0hcPl_yq4jOyOPiA
4,supercuts-san-francisco,pPlsW81ntC2qbF0QOe6pLQ,5.0,_3KTfz0hcPl_yq4jOyOPiA


In [129]:
rev_user = revs_df.join(user_cl_n, on = 'user_id', how = 'right')

In [131]:
rev_user.shape

(12563, 20)

In [139]:
rev_user_biz_gr = rev_user.merge(Y_gr, on = 'alias')

In [140]:
rev_user_biz_rf = rev_user.merge(Y_rf, on = 'alias')

In [141]:
rev_user_biz_gr.head(3)

,alias,id,rating_x,user_id,ucl_t0,ucl_t1,ucl_t2,ucl_t3,ucl_t4,ucl_t5,...,cl_t8,cl_t9,cl_t10,cl_t11,cl_t12,cl_t13,cl_t14,cl_t16,cl_t17,rating_y
0,srivilai-thai-cuisine-seattle,ErsYo_9v8--feFCy92tymQ,5.0,_3KTfz0hcPl_yq4jOyOPiA,True,True,False,False,False,False,...,0.002361,0.006865,0.000325,0.011974,0.000438,0.003589,0.0,0.030197,0.003987,4.5
1,srivilai-thai-cuisine-seattle,_oJOAI1Agf5uSbQrT6gS0A,5.0,iJjKL0KxMZJCb8MmAsERvA,True,True,False,False,False,False,...,0.002361,0.006865,0.000325,0.011974,0.000438,0.003589,0.0,0.030197,0.003987,4.5
2,srivilai-thai-cuisine-seattle,OaaR9v40Id6czlxw6vicTg,4.0,3DLg6FHxYaueFO-47can0w,True,True,False,False,False,False,...,0.002361,0.006865,0.000325,0.011974,0.000438,0.003589,0.0,0.030197,0.003987,4.5


In [143]:
with open('thres_72_gr', 'rb') as f:
    th_gr72 = pickle.load(f)
with open('thres_72_rf','rb') as f:
    th_rf72 = pickle.load(f )

In [192]:
with open('thres_30_gr', 'rb') as f:
    th_gr30 = pickle.load(f)
with open('thres_30_rf','rb') as f:
    th_rf30 = pickle.load(f )

In [193]:
def predict_rating(row, colClust_n, thres):
    r = False
    for col in colClust_n:
        u_col = 'u'+col
        if row[u_col] and row[col] >= thres[col]:
            r = True
    return r

In [194]:
pred_gr = rev_user_biz_gr.apply(predict_rating, axis=1, args = (colClust_n, th_gr30))

In [195]:
pred_rf = rev_user_biz_rf.apply(predict_rating, axis=1, args = (colClust_n, th_rf30))

In [154]:
act = (rev_user_biz_gr['rating_x'].astype(float) <3)

In [188]:
base35 = (rev_user_biz_gr['rating_y'].astype(float) <3.5)
base4 = (rev_user_biz_gr['rating_y'].astype(float) <4)

In [190]:
recall_score(act, base4), accuracy_score(act, base4), precision_score(act, base4)

(0.570230607966457, 0.6651034482758621, 0.21233411397345825)

In [196]:
recall_score(act, base35), recall_score(act, pred_gr), recall_score(act, pred_rf)

(0.21174004192872117, 0.8805031446540881, 0.7840670859538784)

In [197]:
accuracy_score(act, base35), accuracy_score(act, pred_gr), accuracy_score(act, pred_rf)

(0.8226206896551724, 0.31613793103448273, 0.43310344827586206)

In [198]:
precision_score(act, base35), precision_score(act, pred_gr), precision_score(act, pred_rf)

(0.27445652173913043, 0.1477832512315271, 0.16079105760963028)

In [199]:
act.mean(), base35.mean(), pred_gr.mean(), pred_rf.mean()

(0.13158620689655173, 0.10151724137931034, 0.784, 0.6416551724137931)